In [23]:
test_path = '/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet'
train_path = '/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet'
model_path = '/kaggle/working/LaBSE_model'

In [6]:
#from sentence_transformers import SentenceTransformer

#model = SentenceTransformer("sentence-transformers/LaBSE") # Or any other model
#model.save("./LaBSE_model") # Save to a local directory
#print("Model saved locally!")

Model saved locally!


In [28]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import os

In [10]:
train_df = pd.read_parquet(train_path)
print(train_df.head())

                                                  id  \
0  00007cff95d7f7974642a785aca248b0f26e60d3312fac...   
1  00010ed04b536f56ebe43eef1100c13906abea12bf9855...   
2  0003800d510e38803efba5ceaec122bc66408fe367b0be...   
3  00072026c68f5418ef2da238394e418ce72a534b9b22d5...   
4  0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...   

                                              prompt  \
0                                 vieš po Slovensky?   
1  You will be given a piece of news. Analyze it ...   
2  Dört basamaklı, rakamları birbirinden ve sıfır...   
3  현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...   
4                                  Please be boring    

                                          response_a  \
0   Áno, hovorím po slovensky. Ako vám môžem pomôcť?   
1  Let's break down the news and analyze it accor...   
2  Bu soruyu çözmek için, verilen koşulları adım ...   
3  죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...   
4  Alright, I'll be as boring as possible.\n\n

In [ ]:
print(train_df.info())
print(train_df.describe())

In [11]:
test_df = pd.read_parquet(test_path)
print(test_df.head())

        id                                             prompt  \
0   327228  Caso Clínico: Un hombre de 70 años con anteced...   
1  1139415   Peel Company received a cash dividend from a ...   
2  1235630  Há um grave problema com o relógio da torre da...   

                                          response_a  \
0  **Diagnóstico Diferencial de Anemia en Pacient...   
1  The correct answer is **(a) No   No**. Here's ...   
2  Dois problemas interessantes!\n\n**Problema 1:...   

                                          response_b  scored  
0  Basándonos en el caso clínico presentado, pode...   False  
1  The correct answer is **(a) No No**. Here's wh...   False  
2  Vamos resolver os dois problemas em sequência....   False  


In [ ]:
print(test_df.info())
print(test_df.describe())

In [31]:
if os.path.isdir(model_path):
    print("Model Exists in the path ", model_path)
    model = SentenceTransformer(model_path)
    print("Model Loaded")

else:
    print("Model Not loaded (offline)!")

print("Current working directory !!", os.getcwd())

Model Exists in the path  /kaggle/working/LaBSE_model
Model Loaded
Current working directory !! /kaggle/working


In [24]:
# Initialize model (downloads pretrained weights)
#model = SentenceTransformer("sentence-transformers/LaBSE")

#model = SentenceTransformer(model_path)

In [ ]:
#from transformers import AutoModel, AutoTokenizer

#model_path = '/root/.cache/torch/sentence_transformers/sbert.net_models_LaBSE/'
#model = AutoModel.from_pretrained(model_path)

In [13]:
def predict_winners(df, batch_size=5000):
    # Prepare batches
    num_batches = np.ceil(len(df) / batch_size)
    predictions = []

    for batch_idx in range(int(num_batches)):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        batch = df.iloc[batch_start:batch_end]

        # Encode prompts and responses in batches
        prompts_embeddings = model.encode(batch['prompt'].tolist())
        responses_a_embeddings = model.encode(batch['response_a'].tolist())
        responses_b_embeddings = model.encode(batch['response_b'].tolist())

        # Compute cosine similarities for the batch
        sim_a = cosine_similarity(prompts_embeddings, responses_a_embeddings)
        sim_b = cosine_similarity(prompts_embeddings, responses_b_embeddings)

        #You need to extract the diagonal elements since you are comparing one-to-one pairs
        sim_a_diag = np.diag(sim_a)
        sim_b_diag = np.diag(sim_b)

        # Determine winners for the batch
        batch_predictions = ['model_a' if sim_a_diag[i] > sim_b_diag[i] else 'model_b' for i in range(len(batch))]
        predictions.extend(batch_predictions)

    return predictions

In [15]:
# Apply the batch prediction function
train_df['predicted_winner'] = predict_winners(train_df)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/108 [00:00<?, ?it/s]

Batches:   0%|          | 0/108 [00:00<?, ?it/s]

Batches:   0%|          | 0/108 [00:00<?, ?it/s]

In [16]:
# Assuming 'train_df' already contains a 'winner' column with the ground truth labels
# and 'predicted_winner' column with the predictions from 'predict_winners'

# Calculate accuracy
accuracy = (train_df['winner'] == train_df['predicted_winner']).mean()
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 51.57%


In [17]:
# Apply the batch prediction function to the test set
test_df['predicted_winner'] = predict_winners(test_df)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# Create a DataFrame for submission
submission_df = test_df[['id', 'predicted_winner']]
submission_df.columns = ['id', 'winner']  # Rename columns to match the required submission format

# Save the DataFrame to a CSV file, without the index
submission_df.to_csv('submission.csv', index=False)

In [19]:
# Print the first few lines of the submission file to confirm
print(submission_df.head())

        id   winner
0   327228  model_a
1  1139415  model_a
2  1235630  model_b
